<a href="https://colab.research.google.com/github/kwang517/Data-2/blob/MyApp/Midterm_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl ipecho.net/plain

34.148.143.105

In [2]:
!pip install streamlit --quiet
!pip install pyngrok --quiet
!pip install hvplot --quiet
!pip install install psutil py-cpuinfo
!pip install codecarbon
!pip freeze > requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 13.5 MB/s eta 0:00:00


In [3]:
%%writefile app.py
import streamlit as st
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import time

app_mode = st.sidebar.selectbox('Selectpage:',['01 Introduction','02 Visualization', '03 Prediction'])
if app_mode == '01 Introduction':
  image_movie = Image.open('movies.jpg')
  st.image(image_movie, width=700)

  st.title("A quantitative observation: What leads to a movie's global success?")

  st.sidebar.header("Dashboard")
  st.sidebar.markdown("---")
  app_mode = st.sidebar.selectbox('Select Page',['Introduction'])

  st.markdown("##### Key Objectives")
  st.markdown("We aim to investigate the relationship between different factors of movie performance and how it contributes to their profitability as measured by their worldwide gross")
  st.markdown("Our data explores the world's top 500 movies by production budget, and these production budgets are not inflation adjusted")
  st.markdown("The costs and grosses are measured in USD.")

  df = pd.read_csv("top-500-movies.csv")

  head = st.radio('View the top 10 movies by production expenses or View the last 10 movies by production expenses', ('Top 10', 'Last 10'))
  if head == 'Top 10':
    st.dataframe(df.head(10))
  else:
    st.dataframe(df.tail(10))

  st.markdown("##### Key Variables")
  st.markdown("- opening_weekend performance")
  st.markdown("- number of theatres released")
  st.markdown("- production cost")
  st.markdown("- domestic gross")
  st.markdown("- genre")

  st.markdown("**opening weekend performance indicates how much revenue the movie makes by the first weekend of release.")
  st.markdown("**number of theatres released indicates how many theatres the movie gets released in, measuring the spread of the movie distribution internationally. If the movie is aired in greater amount of movie theatres, it may have a chance of expanding its audience base, therefore generating a larger return")
  st.markdown("**The production cost of each movie indicates the total money spent throughout the production, this may vary based on the different needs of production. ")
  st.markdown("**Domestic gross indicates the returns generated by the movie within its original nation of release. For instance, the domestic gross of Star Wars EPVII:The Force Awakens measures the total revenue generated in United States. ")
  st.markdown("**The genre of the movie included: Action,Adventure,Musical,Western,Drama,Thriller/Suspense,Comedy, Horror, Black Comedy,Romantic Comedy")
  st.markdown("For movie productions in the future, it is critical to measure from multiple perspectives. ")

  st.markdown("### Description of Data")
  st.dataframe(df.describe())
  st.markdown("Descriptions for all quantitative data by:")

  st.markdown("Count")
  st.markdown("Mean")
  st.markdown("Standard Deviation")
  st.markdown("Minimum")
  st.markdown("Quartiles:25%,50%,70%")
  st.markdown("Maximum")
  st.markdown("note: The rank and year of release in the description are not relevant quantitative data for observation, one shall ignore the calculations.")

  st.markdown("### Missing Values")
  st.markdown("Null or NaN values.")

  dfnull = df.isnull().sum()/len(df)*100
  totalmiss = dfnull.sum().round(2)
  st.write("Percentage of total missing values:",totalmiss)
  st.write(dfnull)
  if totalmiss == 0.0:
    st.success("We do not exprience any missing values which is the ideal outcome of our data. We can proceed with higher accuracy in our further prediction.")
  else:
    st.warning("Poor data quality due to greater than 30 percent of missing value.")
    st.markdown(" > Theoretically, 25 to 30 percent is the maximum missing values are allowed, there's no hard and fast rule to decide this threshold. It can vary from problem to problem.")

  st.markdown("### Completeness")
  st.markdown(" The ratio of non-missing values to total records in dataset and how comprehensive the data is.")

  st.write("Total data length:", len(df))
  nonmissing = (df.notnull().sum().round(2))
  completeness= round(sum(nonmissing)/len(df),2)

  st.write("Completeness ratio:",completeness)
  st.write(nonmissing)
  if completeness >= 0.80:
    st.success("We have completeness ratio greater than 0.85, which is good. It shows that the vast majority of the data is available for us to use and analyze. ")
  else:
    st.success("Poor data quality due to low completeness ratio( less than 0.85).")

elif app_mode == '02 Visualization':
  df = pd.read_csv("top-500-movies.csv")
  list_variables = df.columns

  st.title("Visualization")
  symbols = st.multiselect("Select two variables",list_variables,default=['opening_weekend','worldwide_gross'])
  width1 = st.sidebar.slider("plot width", 1, 25, 10)
  #symbols = st.multiselect("", list_variables, list_variables[:5])
  tab1, tab2= st.tabs(["Line Chart & Bar Chart ","📈 Correlation"])

  tab1.subheader("Cinematic Success Patterns: A Visual Exploration of Performance Metrics ")
  st.line_chart(data=df, x=symbols[0],y=symbols[1], width=0, height=0, use_container_width=True)
  st.write(" ")
  st.bar_chart(data=df, x=symbols[0], y=symbols[1], use_container_width=True)

  tab2.subheader("Correlation Matrix: the Relationships Among Movie Success Variables")
  fig,ax = plt.subplots(figsize=(width1, width1))
  sns.heatmap(df.corr(),cmap= sns.cubehelix_palette(8),annot = True, ax=ax)
  tab2.write(fig)

  st.write(" ")
  st.write(" ")
  st.markdown("### Pairplot")

  df2 = df[[list_variables[0],list_variables[1],list_variables[2],list_variables[3],list_variables[4]]]
  fig3 = sns.pairplot(df2)
  st.pyplot(fig3)


elif app_mode == '03 Prediction':
    import streamlit as st
    import matplotlib.pyplot as plt
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    import numpy as np
    import time

    # Load data
    df = pd.read_csv('top-500-movies.csv')

    # Title
    st.title("Movies Grosses Data Analysis - Prediction")

    # Sidebar selections
    prediction_type = st.sidebar.selectbox('Select Type of Prediction', ['Linear Regression'])
    list_variables = ['production_cost', 'domestic_gross', 'worldwide_gross', 'opening_weekend', 'theaters']
    select_variable = st.sidebar.selectbox('Select Variable to Predict', list_variables)
    train_size = st.sidebar.number_input("Train Set Size", min_value=0.00, step=0.01, max_value=1.00, value=0.80)

    # Feature selection
    output_multi = st.multiselect("Select Explanatory Variables", list_variables, default=['opening_weekend', 'domestic_gross'])

    # Splitting data
    x = df[output_multi]
    y = df[select_variable]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=train_size)

    # Scaling data
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    # Linear Regression
    if prediction_type == 'Linear Regression':
        lr_start_time = time.time()
        lr_model = LinearRegression()
        lr_model.fit(x_train_scaled, y_train)
        lr_pred = lr_model.predict(x_test_scaled)
        lr_end_time = time.time()
        lr_execution_time = lr_end_time - lr_start_time

        # Round up the predicted values to the nearest whole number
        lr_pred_rounded = np.ceil(lr_pred)

        # Results
        st.write("Execution time:", lr_execution_time, "seconds")

        col1, col2 = st.columns(2)
        col1.subheader("Feature Columns top 25")
        col1.write(x.head(25))

        # Creating a DataFrame for comparing actual and predicted values
        y_test_df = pd.DataFrame(y_test).reset_index(drop=True)
        predictions_df = pd.DataFrame(lr_pred_rounded, columns=['Predicted_Rounded'])
        comparison_df = pd.concat([y_test_df, predictions_df], axis=1)
        comparison_df.columns = [select_variable, 'Predicted_Rounded']

        st.subheader("Comparison of Actual and Predicted Values")
        st.write(comparison_df)

    # Plotting Actual vs. Predicted values
    rolling_window_size = 50  # The window size can be adjusted to your preference
    comparison_df['Actual_Rolling'] = comparison_df[select_variable].rolling(rolling_window_size).mean()
    comparison_df['Predicted_Rolling'] = comparison_df['Predicted_Rounded'].rolling(rolling_window_size).mean()

    # Plotting smoothed Actual vs. Predicted values as lines
    fig, ax = plt.subplots()
    ax.plot(comparison_df.index, comparison_df['Actual_Rolling'], label='Actual', color='blue', linestyle='-')
    ax.plot(comparison_df.index, comparison_df['Predicted_Rolling'], label='Predicted', color='red', linestyle='-')

    # Set the x-axis label, y-axis label, and the title of the graph
    ax.set_xlabel('Index')  # Use 'Index' if there is no date or specific x-axis label
    ax.set_ylabel(select_variable)  # Use the name of the target variable for the y-axis label
    ax.set_title('Comparison of Actual and Predicted Values - Smoothed')  # Title of the graph with indication of smoothing

    # Add a legend to distinguish between actual and predicted lines
    ax.legend()

    # note
    st.text("Note: '1e8' on the y-axis represents a scale factor of $100,000,000.")
    # Display the plot in Streamlit
    ax.set_xlim(right=200)
    st.pyplot(fig)

    # Calculating metrics
    mae = mean_absolute_error(y_test, lr_pred_rounded)
    r2 = r2_score(y_test, lr_pred_rounded)

    # Displaying metrics
    st.subheader('Results')
    st.write(f"Mean Absolute Error: {mae}")
    st.write(f"R^2 Score: {r2}")

Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

[##................] - fetchMetadata: sill resolveWithNewModule yargs-parser@20


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.148.143.105:8501

npx: installed 22 in 6.487s
your url is: https://wise-drinks-sin.loca.lt
2024-03-04 14:09:15.114 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 535, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 151, in <module>
    scaler.fit(x_train)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py", line 824, in fit
    return self.partial_fit(X, y, sample_weight)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py", line 861, in partial_fit
    X = self._validate_data(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 565, in _validate_data
    X = check_array(X